# 1. Frame the Problem and Look at the Big Picture

Write a function that can shift an MNIST image in any direction (left, right, up,
or down) by one pixel. Then, for each image in the training set, create four shifted
copies (one per direction) and add them to the training set. Finally, train your
best model on this expanded training set and measure its accuracy on the test set.
You should observe that your model performs even better now! This technique of artificially growing the training set is called data augmentation or training set
expansion.

Problem is of Supervised ML

# 2. Get the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def GetData():
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784')
    X, y = mnist['data'], mnist['target'].astype(int)
    shuffle_index = np.random.permutation(70000)
    X_train, X_test, y_train, y_test = X.iloc[shuffle_index[:60000]], X.iloc[shuffle_index[60000:]], y[shuffle_index[:60000]], y[shuffle_index[60000:]]
    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = GetData()

# 3. Function to Shift the images

In [5]:
def ShiftImage(image, shift):
    if(shift== 'l' or shift == 'r'  or shift == 'u' or shift == 'd'):
        shifted_image = None
        if(shift == 'u'):
            shifted_image = np.concatenate((image[:756], np.zeros(28)))
        elif(shift == 'd'):
            shifted_image = np.concatenate((np.zeros(28), image[28:]))
        elif(shift == 'r'):
            shifted_image = []
            for i in range(0, 784):
                if(i!=0 and i%28 != 0):
                    shifted_image.append(image[i])
                else:
                    shifted_image.append(0)
            shifted_image = np.array(shifted_image)
        elif(shift == 'l'):
            shifted_image = []
            for i in range(0, 784):
                if((i+1)%28 != 0):
                    shifted_image.append(image[i])
                else:
                    shifted_image.append(0)
            shifted_image = np.array(shifted_image)
        return shifted_image
    else:
        print(f"Please provide valid argument for shift parameter. Parameters allowed are 'l' (left), 'r' (right), 'u' (up), 'd' (down)")
        return

## 3.1 Try with some digit

In [ ]:
import matplotlib
some_digit = np.array(X_train.iloc[36000])
some_digit_image = some_digit.reshape(28,28)
plt.imshow(some_digit_image, matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')

In [ ]:
# Left Shift
left_shift_some_digit = ShiftImage(some_digit, 'l')
left_shift_some_digit_image = left_shift_some_digit.reshape(28,28)
plt.imshow(left_shift_some_digit_image, matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')

In [ ]:
# Right Shift
right_shift_some_digit = ShiftImage(some_digit, 'r')
right_shift_some_digit_image = right_shift_some_digit.reshape(28,28)
plt.imshow(right_shift_some_digit_image, matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')

In [ ]:
# Up Shift
up_shift_some_digit = ShiftImage(some_digit, 'u')
up_shift_some_digit_image = up_shift_some_digit.reshape(28,28)
plt.imshow(up_shift_some_digit_image, matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')

In [ ]:
# Down Shift
down_shift_some_digit = ShiftImage(some_digit, 'd')
down_shift_some_digit_image = down_shift_some_digit.reshape(28,28)
plt.imshow(down_shift_some_digit_image, matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')

# 4. Training Set Expansion

In [40]:
def TrainingSetExpander(X_train, y_train):
    # Expansion of X_train
    X_train_columns = X_train.columns.values
    X_train = np.array(X_train)
    data_to_be_concat_in_X_train = []
    for shift in ['u', 'r', 'd', 'l']:
        one_shift_data = []
        for i in range(len(X_train)):
            image_instance = X_train[i]
            shifted_image = ShiftImage(image_instance, shift)
            one_shift_data.append(list(shifted_image))
        data_to_be_concat_in_X_train = data_to_be_concat_in_X_train + one_shift_data
    final_X_train = np.concatenate((X_train, np.array(data_to_be_concat_in_X_train)))
    final_X_train = pd.DataFrame(final_X_train, columns=X_train_columns)

    # Expansion of y_train
    y_train = list(np.array(y_train))
    final_y_train = []
    for i in range(5):
        final_y_train = final_y_train + y_train
    final_y_train = np.array(final_y_train)

    return final_X_train, final_y_train

In [36]:
dtbcixt = []
for shift in ['u', 'r']:
    osd = []
    for i in range(2):
        ii = np.array(X_train.iloc[i])
        si = ShiftImage(ii, shift)
        osd.append(list(si))
    dtbcixt = dtbcixt + osd
fxt = np.concatenate((np.array(X_train.iloc[:2]), np.array(dtbcixt)))
fxt = pd.DataFrame(fxt, columns=X_train.columns)

In [41]:
X_train_expanded, y_train_expanded = TrainingSetExpander(X_train, y_train)

# 5. Training the Model

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf.fit(X_train_expanded, y_train_expanded)

KNeighborsClassifier(n_neighbors=4, weights='distance')

# 6. Testing the Model

In [46]:
from sklearn.metrics import accuracy_score
y_predicted_test = knn_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_predicted_test)
print(f"Test accuracy is: {test_accuracy}")

Test accuracy is: 0.9735
